### Cal-CRAI Metric Calculation for: Economic Health
* Hachman Index

In [1]:
import pandas as pd
import os
import sys
import numpy as np

# suppress pandas purely educational warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import pull_csv_from_directory, upload_csv_aws, filter_counties
from scripts.utils.write_metadata import append_metadata

In [2]:
# Define a function to display data for a specific county
def display_county_data(df, county_name):
    county_data = df[df['County'] == county_name]
    if county_data.empty:
        print(f"No data found for {county_name}")
    else:
        print(f"Data for {county_name}:")
        display(county_data)
        

In [3]:
# pull csv from aws
bucket_name = 'ca-climate-index'
aws_dir = '2a_subset/society_economy/economic_health/bureau_labor_statistics/employment_hachman_index/'

pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

aws_dir = '1_pull_data/society_economy/economic_health/'
pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

Saved DataFrame as 'employment_data_hachman_subset.csv'
Saved DataFrame as 'ACSDT5Y2022.B19083-Column-Metadata.csv'
Saved DataFrame as 'ACSDT5Y2022.B19083-Data.csv'
Saved DataFrame as 'ACSST5Y2022.S1901-Column-Metadata.csv'


/Users/victoriaford/Desktop/carb-climate-index/scripts/data_metric_calc/../../scripts/utils/file_helpers.py:100: DtypeWarning: Columns (2,3,34,35,66,67,98,99) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_data)


Saved DataFrame as 'ACSST5Y2022.S1901-Data.csv'


/Users/victoriaford/Desktop/carb-climate-index/scripts/data_metric_calc/../../scripts/utils/file_helpers.py:100: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_data)


Saved DataFrame as 'allhlcn22.csv'
Saved DataFrame as 'data_layout.csv'


In [4]:
hachman_data = pd.read_csv('employment_data_hachman_subset.csv')

In [5]:
county_tract = "s3://ca-climate-index/0_map_data/ca_tract_county_population_2022.csv"
ca_county_tract = pd.read_csv(county_tract)
ca_county_tract = ca_county_tract.rename(columns={'Census Tract': 'GEO_ID'})
ca_county_tract = ca_county_tract.drop(columns={'Unnamed: 0', 'COUNTYFP', 'County', 'Total Population 2021'})

### Hachman Index Metric
$$
HI = \frac{1}{Σ_j \ \left(\frac{E_si}{E_ri}\right) \cdot E_si}
$$

{E_si} is share of area economic indicator in industry i -- county level <br>
{E_ri} is share of regions economic indicator in industry i -- state level

In [6]:
hachman_data.head()

,Area\nCode,St,Cnty,Own,NAICS,Year,Qtr,Area Type,St Name,Area,Ownership,Industry,Annual Average Status Code,Annual Average Establishment Count,Annual Average Employment,Annual Total Wages,Annual Average Weekly Wage,Annual Average Pay,Employment Location Quotient Relative to U.S.,Total Wage Location Quotient Relative to U.S.
0,6000,6,0,0,10,2022,A,State,California,California -- Statewide,Total Covered,"10 Total, all industries",NaN,"1,706,672","17,903,539","1,511,706,499,616","1,624","84,436",1.00,1.00
1,6000,6,0,1,10,2022,A,State,California,California -- Statewide,Federal Government,"10 Total, all industries",NaN,"3,286","247,795","23,655,167,888","1,836","95,463",0.73,0.62
2,6000,6,0,2,10,2022,A,State,California,California -- Statewide,State Government,"10 Total, all industries",NaN,"13,784","492,218","47,749,398,000","1,866","97,009",0.91,1.00
3,6000,6,0,3,10,2022,A,State,California,California -- Statewide,Local Government,"10 Total, all industries",NaN,"19,344","1,724,972","135,804,850,033","1,514","78,729",1.04,1.11
4,6000,6,0,5,10,2022,A,State,California,California -- Statewide,Private,"10 Total, all industries",NaN,"1,670,259","15,438,555","1,304,497,083,695","1,625","84,496",1.01,1.00


Taking a look at the entries within area
* separating California entries as we will need that data also
* getting rid of all 'County, California' portions of each entry

In [7]:
# Get unique entries in 'Column1'
unique_entries = hachman_data['Area'].unique()
#print(unique_entries)

In [8]:
hachman_data_cleaned = hachman_data[['Area', 'Industry', 'Annual Average Employment']]
# Remove any mention of 'county' within the legalAgencyName column
hachman_data_cleaned['Area'] = hachman_data_cleaned['Area'].str.replace(' -- Statewide', '', case=False)
hachman_data_cleaned['Area'] = hachman_data_cleaned['Area'].str.replace(' County, California', '', case=False)

unique_entries = hachman_data_cleaned['Area'].unique()
hachman_data_cleaned = hachman_data_cleaned.rename(columns={'Area':'County'})

/var/folders/6g/b2z199kx72x11z3xdvsz3ytr0000gn/T/ipykernel_44131/2878593510.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hachman_data_cleaned['Area'] = hachman_data_cleaned['Area'].str.replace(' -- Statewide', '', case=False)
/var/folders/6g/b2z199kx72x11z3xdvsz3ytr0000gn/T/ipykernel_44131/2878593510.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hachman_data_cleaned['Area'] = hachman_data_cleaned['Area'].str.replace(' County, California', '', case=False)


In [9]:
hachman_data_cleaned

,County,Industry,Annual Average Employment
0,California,"10 Total, all industries","17,903,539"
1,California,"10 Total, all industries","247,795"
2,California,"10 Total, all industries","492,218"
3,California,"10 Total, all industries","1,724,972"
4,California,"10 Total, all industries","15,438,555"
...,...,...,...
1069,"Unknown Or Undefined, California",1024 Professional and business services,"236,341"
1070,"Unknown Or Undefined, California",1025 Education and health services,"30,314"
1071,"Unknown Or Undefined, California",1026 Leisure and hospitality,"10,276"
1072,"Unknown Or Undefined, California",1027 Other services,0


In [10]:
display_county_data(hachman_data_cleaned, 'Alameda')

Data for Alameda:


,County,Industry,Annual Average Employment
18,Alameda,"10 Total, all industries","787,020"
19,Alameda,"10 Total, all industries","8,304"
20,Alameda,"10 Total, all industries","25,618"
21,Alameda,"10 Total, all industries","70,792"
22,Alameda,"10 Total, all industries","682,306"
23,Alameda,101 Goods-producing,"146,054"
24,Alameda,1011 Natural resources and mining,959
25,Alameda,1012 Construction,"47,512"
26,Alameda,1013 Manufacturing,"97,583"
27,Alameda,102 Service-providing,"536,253"


There are multiple entries of '10 Total, all industries' within the Industry column for each county (and CA as a whole)
* the first entry per county is the largest and sum of all others, indicating total employment for that county
* the code below retains the first/largest number of this bunch and makes a new df containing these values
which will be used in the Hachman index further below

In [11]:
df = pd.DataFrame(hachman_data_cleaned)

# Filter for '10 Total, all industries' in the 'Industry' column
df_filtered = df[df['Industry'] == '10 Total, all industries']

# Remove commas and convert 'Annual Average Employment' to numeric
df_filtered['Total County Employment'] = df_filtered['Annual Average Employment'].str.replace(',', '').astype(float)

# Group by Area and get the row with the max 'Annual Average Employment'
max_area_values = df_filtered.loc[df_filtered.groupby('County')['Total County Employment'].idxmax()]
max_area_values = max_area_values.drop(columns=['Industry','Annual Average Employment'])
max_area_values

/var/folders/6g/b2z199kx72x11z3xdvsz3ytr0000gn/T/ipykernel_44131/932078627.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Total County Employment'] = df_filtered['Annual Average Employment'].str.replace(',', '').astype(float)


,County,Total County Employment
18,Alameda,787020.0
36,Alpine,668.0
52,Amador,12114.0
70,Butte,78536.0
88,Calaveras,10626.0
0,California,17903539.0
106,Colusa,8901.0
124,Contra Costa,367582.0
142,Del Norte,8104.0
160,El Dorado,57308.0


Now that we have a new df with the total employments, we can get rid of those entries within our main df

In [12]:
# create a Boolean mask for the rows to remove
mask = hachman_data_cleaned['Industry'] == '10 Total, all industries'
# select all rows except the ones that contain '10 Total, all industries' which are state and county employment totals, not needed
hachman_data_cleaned = hachman_data_cleaned[~mask]

Running our county function filter to separate CA counties and other entries

In [13]:
filtered_hachman_data, omitted_data = filter_counties(hachman_data_cleaned, 'County')

print('Counties kept:', len(filtered_hachman_data))
print('Omitted data entries:', len(omitted_data))

Counties kept: 749
Omitted data entries: 26


In [14]:
filtered_hachman_data.rename(columns={'Annual Average Employment': 'Industry Employed County'}, inplace=True)
filtered_hachman_data

/var/folders/6g/b2z199kx72x11z3xdvsz3ytr0000gn/T/ipykernel_44131/2789166438.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_hachman_data.rename(columns={'Annual Average Employment': 'Industry Employed County'}, inplace=True)


,County,Industry,Industry Employed County
23,Alameda,101 Goods-producing,"146,054"
24,Alameda,1011 Natural resources and mining,959
25,Alameda,1012 Construction,"47,512"
26,Alameda,1013 Manufacturing,"97,583"
27,Alameda,102 Service-providing,"536,253"
...,...,...,...
1051,Yuba,1024 Professional and business services,"1,265"
1052,Yuba,1025 Education and health services,"3,615"
1053,Yuba,1026 Leisure and hospitality,"1,893"
1054,Yuba,1027 Other services,194


Looking at the other entries, will isolate for just California data, which will be used in the Hachman calculation

In [15]:
omitted_data

,County,Industry,Annual Average Employment
5,California,101 Goods-producing,"2,685,113"
6,California,1011 Natural resources and mining,"436,965"
7,California,1012 Construction,"912,359"
8,California,1013 Manufacturing,"1,335,789"
9,California,102 Service-providing,"12,753,442"
10,California,"1021 Trade, transportation, and utilities","3,121,930"
11,California,1022 Information,"605,857"
12,California,1023 Financial activities,"841,564"
13,California,1024 Professional and business services,"2,860,674"
14,California,1025 Education and health services,"2,854,971"


In [16]:
# Get rid of the Unknown entries
mask = omitted_data['County'] == 'Unknown Or Undefined, California'
california_employ_data = omitted_data[~mask]
california_employ_data.rename(columns={'Annual Average Employment': 'Industry Employed CA', 'County': 'State'}, inplace=True)

# Now we have a df that holds all CA state employment per industry
california_employ_data

/var/folders/6g/b2z199kx72x11z3xdvsz3ytr0000gn/T/ipykernel_44131/1960439974.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  california_employ_data.rename(columns={'Annual Average Employment': 'Industry Employed CA', 'County': 'State'}, inplace=True)


,State,Industry,Industry Employed CA
5,California,101 Goods-producing,"2,685,113"
6,California,1011 Natural resources and mining,"436,965"
7,California,1012 Construction,"912,359"
8,California,1013 Manufacturing,"1,335,789"
9,California,102 Service-providing,"12,753,442"
10,California,"1021 Trade, transportation, and utilities","3,121,930"
11,California,1022 Information,"605,857"
12,California,1023 Financial activities,"841,564"
13,California,1024 Professional and business services,"2,860,674"
14,California,1025 Education and health services,"2,854,971"


Merge the county and california employment datasets together based on industry so we have county employment and state employment per industry

In [17]:
# Merge county industry employment with state industry employment
merged_data = filtered_hachman_data.merge(california_employ_data, on='Industry')
merged_data

,County,Industry,Industry Employed County,State,Industry Employed CA
0,Alameda,101 Goods-producing,"146,054",California,"2,685,113"
1,Alpine,101 Goods-producing,13,California,"2,685,113"
2,Amador,101 Goods-producing,"1,865",California,"2,685,113"
3,Butte,101 Goods-producing,"11,325",California,"2,685,113"
4,Calaveras,101 Goods-producing,"1,500",California,"2,685,113"
...,...,...,...,...,...
744,Tulare,1029 Unclassified,5,California,"1,970"
745,Tuolumne,1029 Unclassified,0,California,"1,970"
746,Ventura,1029 Unclassified,17,California,"1,970"
747,Yolo,1029 Unclassified,11,California,"1,970"


Merge the result to the total employment values based on county
* result gives us an additional column with total county values

In [18]:
further_merged_data = merged_data.merge(max_area_values, on='County')
further_merged_data

,County,Industry,Industry Employed County,State,Industry Employed CA,Total County Employment
0,Alameda,101 Goods-producing,"146,054",California,"2,685,113",787020.0
1,Alameda,1011 Natural resources and mining,959,California,"436,965",787020.0
2,Alameda,1012 Construction,"47,512",California,"912,359",787020.0
3,Alameda,1013 Manufacturing,"97,583",California,"1,335,789",787020.0
4,Alameda,102 Service-providing,"536,253",California,"12,753,442",787020.0
...,...,...,...,...,...,...
744,Yuba,1024 Professional and business services,"1,265",California,"2,860,674",20314.0
745,Yuba,1025 Education and health services,"3,615",California,"2,854,971",20314.0
746,Yuba,1026 Leisure and hospitality,"1,893",California,"1,929,160",20314.0
747,Yuba,1027 Other services,194,California,"537,319",20314.0


Add another column with the total employment value in California

In [19]:
hachman_denominator = pd.DataFrame(further_merged_data)
hachman_denominator['Total State Employment'] = 17903539.0

hachman_denominator

,County,Industry,Industry Employed County,State,Industry Employed CA,Total County Employment,Total State Employment
0,Alameda,101 Goods-producing,"146,054",California,"2,685,113",787020.0,17903539.0
1,Alameda,1011 Natural resources and mining,959,California,"436,965",787020.0,17903539.0
2,Alameda,1012 Construction,"47,512",California,"912,359",787020.0,17903539.0
3,Alameda,1013 Manufacturing,"97,583",California,"1,335,789",787020.0,17903539.0
4,Alameda,102 Service-providing,"536,253",California,"12,753,442",787020.0,17903539.0
...,...,...,...,...,...,...,...
744,Yuba,1024 Professional and business services,"1,265",California,"2,860,674",20314.0,17903539.0
745,Yuba,1025 Education and health services,"3,615",California,"2,854,971",20314.0,17903539.0
746,Yuba,1026 Leisure and hospitality,"1,893",California,"1,929,160",20314.0,17903539.0
747,Yuba,1027 Other services,194,California,"537,319",20314.0,17903539.0


Calculate the Esi and Eri values
* Esi = (county employment in industry i / total county employment for all industries)
* Eri = (state employment in industry i / total state employment for all industries)

Then we divide Esi by Eri, and multiply by Esi to create our hachman denominator column

In [20]:
hachman_denominator['Industry Employed CA'] = hachman_denominator['Industry Employed CA'].str.replace(',', '').astype(float)
hachman_denominator['Industry Employed County'] = hachman_denominator['Industry Employed County'].str.replace(',', '').astype(float)

In [21]:
# county score per industry -- fraction of county employment in industry
hachman_denominator['county_industry_frac'] = hachman_denominator['Industry Employed County'] / hachman_denominator['Industry Employed CA']

# state score per industry -- fraction of state employment in industry
hachman_denominator['state_industry_frac'] = hachman_denominator['Total County Employment'] / hachman_denominator['Total State Employment']
hachman_denominator

,County,Industry,Industry Employed County,State,Industry Employed CA,Total County Employment,Total State Employment,county_industry_frac,state_industry_frac
0,Alameda,101 Goods-producing,146054.0,California,2685113.0,787020.0,17903539.0,0.054394,0.043959
1,Alameda,1011 Natural resources and mining,959.0,California,436965.0,787020.0,17903539.0,0.002195,0.043959
2,Alameda,1012 Construction,47512.0,California,912359.0,787020.0,17903539.0,0.052076,0.043959
3,Alameda,1013 Manufacturing,97583.0,California,1335789.0,787020.0,17903539.0,0.073053,0.043959
4,Alameda,102 Service-providing,536253.0,California,12753442.0,787020.0,17903539.0,0.042048,0.043959
...,...,...,...,...,...,...,...,...,...
744,Yuba,1024 Professional and business services,1265.0,California,2860674.0,20314.0,17903539.0,0.000442,0.001135
745,Yuba,1025 Education and health services,3615.0,California,2854971.0,20314.0,17903539.0,0.001266,0.001135
746,Yuba,1026 Leisure and hospitality,1893.0,California,1929160.0,20314.0,17903539.0,0.000981,0.001135
747,Yuba,1027 Other services,194.0,California,537319.0,20314.0,17903539.0,0.000361,0.001135


In [22]:
hachman_denominator['scores'] = (hachman_denominator['county_industry_frac'] / hachman_denominator['state_industry_frac']) * hachman_denominator['county_industry_frac']
hachman_denominator

,County,Industry,Industry Employed County,State,Industry Employed CA,Total County Employment,Total State Employment,county_industry_frac,state_industry_frac,scores
0,Alameda,101 Goods-producing,146054.0,California,2685113.0,787020.0,17903539.0,0.054394,0.043959,0.067306
1,Alameda,1011 Natural resources and mining,959.0,California,436965.0,787020.0,17903539.0,0.002195,0.043959,0.000110
2,Alameda,1012 Construction,47512.0,California,912359.0,787020.0,17903539.0,0.052076,0.043959,0.061692
3,Alameda,1013 Manufacturing,97583.0,California,1335789.0,787020.0,17903539.0,0.073053,0.043959,0.121402
4,Alameda,102 Service-providing,536253.0,California,12753442.0,787020.0,17903539.0,0.042048,0.043959,0.040220
...,...,...,...,...,...,...,...,...,...,...
744,Yuba,1024 Professional and business services,1265.0,California,2860674.0,20314.0,17903539.0,0.000442,0.001135,0.000172
745,Yuba,1025 Education and health services,3615.0,California,2854971.0,20314.0,17903539.0,0.001266,0.001135,0.001413
746,Yuba,1026 Leisure and hospitality,1893.0,California,1929160.0,20314.0,17903539.0,0.000981,0.001135,0.000849
747,Yuba,1027 Other services,194.0,California,537319.0,20314.0,17903539.0,0.000361,0.001135,0.000115


Now we sum the Hachman denominator values together per county
* Group by the county and sum Hachman denominator

In [24]:
hachman_denominator_sum = hachman_denominator.groupby('County')['scores'].sum().reset_index()
hachman_denominator_sum

,County,scores
0,Alameda,0.558020
1,Alpine,0.000725
2,Amador,0.020334
3,Butte,0.056106
4,Calaveras,0.006419
5,Colusa,0.068140
6,Contra Costa,0.254360
7,Del Norte,0.003456
8,El Dorado,0.047930
9,Fresno,0.598389


In [25]:
hachman_denominator_sum['hi'] = 1 / hachman_denominator_sum['scores']
hachman_denominator_sum

,County,scores,hi
0,Alameda,0.558020,1.792052
1,Alpine,0.000725,1378.841819
2,Amador,0.020334,49.179346
3,Butte,0.056106,17.823325
4,Calaveras,0.006419,155.799552
5,Colusa,0.068140,14.675768
6,Contra Costa,0.254360,3.931434
7,Del Norte,0.003456,289.315489
8,El Dorado,0.047930,20.863558
9,Fresno,0.598389,1.671154


Make a new column containing the results from 1/Hachman calculation denominator

In [ ]:
hachman_calculation = pd.DataFrame(denominator_county_sum)
hachman_calculation['hachman_index_value'] = 1 / denominator_county_sum['hachman_denominator']

### Final answer in hachman_calculation df, the numbers are way too small

In [ ]:
hachman_calculation

### The code below is running the calculation steps together

In [ ]:
merged_data = filtered_hachman_data.merge(california_employ_data, on='Industry')
denominator = pd.DataFrame(merged_data)
merged_data['EMP_COUNTY'] = merged_data['EMP_COUNTY'].str.replace(',', '').astype(float)
merged_data['EMP_CA'] = merged_data['EMP_CA'].str.replace(',', '').astype(float)

denominator['emp_county_div_emp_ca'] = (merged_data['EMP_COUNTY'] / merged_data['EMP_CA']) * merged_data['EMP_COUNTY']

pd.set_option('display.float_format', '{:,.2f}'.format)

denominator_county_sum = denominator.groupby('County')['emp_county_div_emp_ca'].sum().reset_index()
denominator_county_sum.head()

hachman_calculation = pd.DataFrame(denominator_county_sum)
hachman_calculation['hachman_index_value'] = 1 / denominator_county_sum['emp_county_div_emp_ca']
hachman_calculation
